# Fairness in healthcare utilization scoring model

<b> Reference - https://nbviewer.jupyter.org/github/IBM/AIF360/blob/master/examples/tutorial_medical_expenditure.ipynb
    </b>
  

### This tutorial demonstrates classification model learning with bias mitigation as a part of a Care Management use case using Medical Expenditure data.

The notebook demonstrates how the AIF 360 toolkit can be used to detect and reduce bias when learning classifiers using a variety of fairness metrics and algorithms . It also demonstrates how explanations can be generated for predictions made by models learnt with the toolkit using LIME.

Classifiers are built using Logistic Regression as well as Random Forests.

Bias detection is demonstrated using several metrics, including disparate impact, average odds difference, statistical parity difference, equal opportunity difference, and Theil index.

Bias alleviation is explored via a variety of methods, including reweighing (pre-processing algorithm), prejudice remover (in-processing algorithm), and disparate impact remover (pre-processing technique).

Data from the [Medical Expenditure Panel Survey](https://meps.ahrq.gov/mepsweb/) is used in this tutorial. See [Section 2](#2.-Data-used) below for more details.


## [1.](#Table-of-Contents) Use case

In order to demonstrate how AIF 360 can be used to detect and mitigate bias in classfier models, we adopt the following use case:

1. a data scientist develops a 'fair' healthcare utilization scoring model with respect to defined protected classes. Fairness may be dictated by legal or government regulations, such as a requirement that additional care decisions be not predicated on factors such as race of the patient.


2. developer takes the model AND performance characteristics / specs of the model (e.g. accuracy, fairness tests, etc. basically the model factsheet) and deploys the model in an enterprise app that prioritizes cases for care management.


3. the app is put into production and starts scoring people and making recommendations. 


4. explanations are generated for each recommendation


5. both recommendations and associated explanations are given to nurses as a part of the care management process. The nurses can evaluate the recommendations for quality and correctness and provide feedback.


6. nurse feedback as well as analysis of usage data with respect to specs of the model w.r.t accuracy and fairness is communicated to AI Ops specialist and LOB user periodically.


7. when significant drift in model specs relative to the model factsheet is observed, the model is sent back for retraining.

## [2.](#Table-of-Contents) Data used

The specific data used is the [2015 Full Year Consolidated Data File](https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-181) as well as the [2016 Full Year Consolidated Data File](https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-192).

The 2015 file contains data from rounds 3,4,5 of panel 19 (2014) and rounds 1,2,3 of panel 20 (2015). The 2016 file contains data from rounds 3,4,5 of panel 20 (2015) and rounds 1,2,3 of panel 21 (2016).

For this demonstration, three datasets were constructed: one from panel 19, round 5 (used for learning models), one from panel 20, round 3 (used for deployment/testing of model - steps); the other from panel 21, round 3 (used for re-training and deployment/testing of updated model).

## [3.](#Table-of-Contents) Training models on original 2015 Panel 19 data

First, load all necessary packages

In [9]:
import sys
sys.path.insert(0, '../')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown, display

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler

# Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing

np.random.seed(1)

In [ ]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
import tensorflow as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

### 3.1. Load data & create splits for learning/validating/testing model

Get the dataset and split into train (70%), test (30%)

In [32]:
(dataset_orig_panel19_train,
 dataset_orig_panel19_test) = MEPSDataset19().split([0.7], shuffle=True)

sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]

unprivileged_groups = [{sens_attr: v} for v in
                       dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                     dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

/Users/dhanush/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/aif360/datasets/standard_dataset.py:121: FutureWarning: outer method for ufunc <ufunc 'equal'> is not implemented on pandas objects. Returning an ndarray, but in the future this will raise a 'NotImplementedError'. Consider explicitly converting the Series to an array with '.array' first.
  priv = np.logical_or.reduce(np.equal.outer(vals, df[attr]))


This function will be used throughout the notebook to print out some labels, names, etc.

In [33]:
def describe(train=None, val=None, test=None):
    if train is not None:
        display(Markdown("#### Training Dataset shape"))
        print(train.features.shape)
    if val is not None:
        display(Markdown("#### Validation Dataset shape"))
        print(val.features.shape)
    display(Markdown("#### Test Dataset shape"))
    print(test.features.shape)
    display(Markdown("#### Favorable and unfavorable labels"))
    print(test.favorable_label, test.unfavorable_label)
    display(Markdown("#### Protected attribute names"))
    print(test.protected_attribute_names)
    display(Markdown("#### Privileged and unprivileged protected attribute values"))
    print(test.privileged_protected_attributes, 
          test.unprivileged_protected_attributes)
    display(Markdown("#### Dataset feature names"))
    print(test.feature_names)

Show 2015 dataset details

In [34]:
describe(dataset_orig_panel19_train, None, dataset_orig_panel19_test)

#### Training Dataset shape

(11081, 138)


#### Test Dataset shape

(4749, 138)


#### Favorable and unfavorable labels

...

['RACE']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['AGE', 'RACE', 'PCS42', 'MCS42', 'K6SUM42', 'REGION=1', 'REGION=2', 'REGION=3', 'REGION=4', 'SEX=1', 'SEX=2', 'MARRY=1', 'MARRY=2', 'MARRY=3', 'MARRY=4', 'MARRY=5', 'MARRY=6', 'MARRY=7', 'MARRY=8', 'MARRY=9', 'MARRY=10', 'FTSTU=-1', 'FTSTU=1', 'FTSTU=2', 'FTSTU=3', 'ACTDTY=1', 'ACTDTY=2', 'ACTDTY=3', 'ACTDTY=4', 'HONRDC=1', 'HONRDC=2', 'HONRDC=3', 'HONRDC=4', 'RTHLTH=-1', 'RTHLTH=1', 'RTHLTH=2', 'RTHLTH=3', 'RTHLTH=4', 'RTHLTH=5', 'MNHLTH=-1', 'MNHLTH=1', 'MNHLTH=2', 'MNHLTH=3', 'MNHLTH=4', 'MNHLTH=5', 'HIBPDX=-1', 'HIBPDX=1', 'HIBPDX=2', 'CHDDX=-1', 'CHDDX=1', 'CHDDX=2', 'ANGIDX=-1', 'ANGIDX=1', 'ANGIDX=2', 'MIDX=-1', 'MIDX=1', 'MIDX=2', 'OHRTDX=-1', 'OHRTDX=1', 'OHRTDX=2', 'STRKDX=-1', 'STRKDX=1', 'STRKDX=2', 'EMPHDX=-1', 'EMPHDX=1', 'EMPHDX=2', 'CHBRON=-1', 'CHBRON=1', 'CHBRON=2', 'CHOLDX=-1', 'CHOLDX=1', 'CHOLDX=2', 'CANCERDX=-1', 'CANCERDX=1', 'CANCERDX=2', 'DIABDX=-1', 'DIABDX=1', 'DIABDX=2', 'JTPAIN=-1', 'JTPAIN=1', 'JTPAIN=2', 'ARTHDX=-1', 'ARTHDX=1', 'ARTHDX=2', 'ARTHTYPE=-1'

Metrics for original data

In [35]:
metric_orig_panel19_train = BinaryLabelDatasetMetric(
        dataset_orig_panel19_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
explainer_orig_panel19_train = MetricTextExplainer(metric_orig_panel19_train)

print(explainer_orig_panel19_train.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.5062355260737379


# Adverserial Debiasing 

In [36]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_panel19_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_panel19_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.133492
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.138635


In [37]:
min_max_scaler = MaxAbsScaler()
dataset_orig_panel19_train.features = min_max_scaler.fit_transform(dataset_orig_panel19_train.features)
dataset_orig_panel19_test.features = min_max_scaler.transform(dataset_orig_panel19_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_panel19_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_panel19_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.133492
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.138635


In [38]:
tf.reset_default_graph()
sess = tf.Session()
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [39]:
debiased_model.fit(dataset_orig_panel19_train)

epoch 0; iter: 0; batch classifier loss: 0.595445; batch adversarial loss: 0.708192
epoch 1; iter: 0; batch classifier loss: 0.343685; batch adversarial loss: 0.685541
epoch 2; iter: 0; batch classifier loss: 0.369257; batch adversarial loss: 0.679864
epoch 3; iter: 0; batch classifier loss: 0.319929; batch adversarial loss: 0.669934
epoch 4; iter: 0; batch classifier loss: 0.408852; batch adversarial loss: 0.657711
epoch 5; iter: 0; batch classifier loss: 0.385198; batch adversarial loss: 0.655831
epoch 6; iter: 0; batch classifier loss: 0.348530; batch adversarial loss: 0.642233
epoch 7; iter: 0; batch classifier loss: 0.312382; batch adversarial loss: 0.661296
epoch 8; iter: 0; batch classifier loss: 0.297410; batch adversarial loss: 0.659129
epoch 9; iter: 0; batch classifier loss: 0.314212; batch adversarial loss: 0.647203
epoch 10; iter: 0; batch classifier loss: 0.291192; batch adversarial loss: 0.653255
epoch 11; iter: 0; batch classifier loss: 0.372638; batch adversarial loss:

In [40]:
dataset_debiasing_test = debiased_model.predict(dataset_orig_panel19_test)

In [41]:
# Metrics for the dataset from model with debiasing

display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_panel19_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Statistical parity difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.821947
Test set: Balanced classification accuracy = 0.700171
Test set: Disparate impact = 0.565905
Test set: Equal opportunity difference = -0.045146
Test set: Average odds difference = -0.040512
Test set: Statistical parity difference = -0.091879
Test set: Theil_index = 0.112782


# Adverserial Debiasing with weighted dataset

In [42]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_panel19_train = RW.fit_transform(dataset_orig_panel19_train)

In [43]:
# Metric for the transformed dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_transf_panel19_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [44]:
tf.reset_default_graph()
sess = tf.Session()
# Learn parameters with debias set to True
weighted_debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [45]:
weighted_debiased_model.fit(dataset_transf_panel19_train)

epoch 0; iter: 0; batch classifier loss: 0.704628; batch adversarial loss: 0.753521
epoch 1; iter: 0; batch classifier loss: 0.363869; batch adversarial loss: 0.694662
epoch 2; iter: 0; batch classifier loss: 0.253869; batch adversarial loss: 0.680370
epoch 3; iter: 0; batch classifier loss: 0.309635; batch adversarial loss: 0.667766
epoch 4; iter: 0; batch classifier loss: 0.303630; batch adversarial loss: 0.663555
epoch 5; iter: 0; batch classifier loss: 0.300757; batch adversarial loss: 0.658754
epoch 6; iter: 0; batch classifier loss: 0.330101; batch adversarial loss: 0.658806
epoch 7; iter: 0; batch classifier loss: 0.352510; batch adversarial loss: 0.684398
epoch 8; iter: 0; batch classifier loss: 0.230057; batch adversarial loss: 0.628915
epoch 9; iter: 0; batch classifier loss: 0.432744; batch adversarial loss: 0.659590
epoch 10; iter: 0; batch classifier loss: 0.255797; batch adversarial loss: 0.635037
epoch 11; iter: 0; batch classifier loss: 0.237745; batch adversarial loss:

In [46]:
dataset_debiasing_test = weighted_debiased_model.predict(dataset_orig_panel19_test)

In [47]:
# Metrics for the dataset from model with debiasing

display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_panel19_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Statistical parity difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.824746
Test set: Balanced classification accuracy = 0.678085
Test set: Disparate impact = 0.626986
Test set: Equal opportunity difference = 0.016688
Test set: Statistical parity difference = -0.063400
Test set: Average odds difference = -0.001990
Test set: Theil_index = 0.118844


In [48]:
## End of Adv Learning

## [10.](#Table-of-Contents) SUMMARY

In [75]:
results = [lr_orig_metrics, lr_transf_metrics,
           lr_transf_metrics_panel20_deploy,
           lr_transf_metrics_panel21_deploy,
           lr_transf_metrics_panel20_test,
           lr_transf_panel20_metrics_panel21_deploy]
debias = pd.Series([''] + ['Reweighing']*5, name='Bias Mitigator')
clf = pd.Series(['Logistic Regression']*6, name='Classifier')
tr = pd.Series(['Panel19']*4 + ['Panel20']*2, name='Training set')
te = pd.Series(['Panel19']*2 + ['Panel20', 'Panel21']*2, name='Testing set')
pd.concat([pd.DataFrame(m) for m in results], axis=0).set_index([debias, clf, tr, te])

,,,,avg_odds_diff,bal_acc,disp_imp,eq_opp_diff,stat_par_diff,theil_ind
Bias Mitigator,Classifier,Training set,Testing set,,,,,,
,Logistic Regression,Panel19,Panel19,-0.205706,0.775935,0.426176,-0.222779,-0.261207,0.092122
Reweighing,Logistic Regression,Panel19,Panel19,-0.015104,0.753893,0.751755,-0.003518,-0.087196,0.096575
Reweighing,Logistic Regression,Panel19,Panel20,0.007135,0.731136,0.805724,0.030262,-0.059602,0.101910
Reweighing,Logistic Regression,Panel19,Panel21,-0.014340,0.737916,0.744126,-0.004405,-0.081262,0.099420
Reweighing,Logistic Regression,Panel20,Panel20,0.004045,0.731345,0.825168,0.041814,-0.069257,0.096305
Reweighing,Logistic Regression,Panel20,Panel21,-0.010875,0.734998,0.809590,-0.004093,-0.075011,0.095536
